In [1]:
import cv2


In [2]:
import mediapipe as mp
import pyautogui
pyautogui.FAILSAFE = False
import util
from pynput.mouse import Button, Controller
mouse = Controller()

ModuleNotFoundError: No module named 'util'

In [ ]:
screen_width, screen_height = pyautogui.size()

In [ ]:
mouse = Controller()

In [ ]:
mpHands=mp.solutions.hands
hands=mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)

In [ ]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks= processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]

    return None

In [ ]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y * screen_height)

        # Ensure the coordinates are within bounds
        x = max(1, min(x, screen_width - 1))
        y = max(1, min(y, screen_height - 1))
        
        # print(f"Moving mouse to: x={x}, y={y}")  # Debug print statement
        pyautogui.moveTo(x, y)


In [ ]:
def is_left_click(landmarks_list,thumb_index_dist):
    return (util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and
           util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])>80 and
           thumb_index_dist>50)

In [ ]:
def is_right_click(landmarks_list,thumb_index_dist):
    return((util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])<50 and
           util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])>80 and
           thumb_index_dist>50))

In [ ]:
def is_double_click(landmarks_list,thumb_index_dist):
    return((util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and
           util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])<50 and
           thumb_index_dist>50))

In [ ]:
def is_screenshot(landmarks_list,thumb_index_dist):
    return(util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])<50 and
           util.get_angle(landmarks_list[9],landmarks_list[10],landmarks_list[12])<50 and
           thumb_index_dist<50)

In [ ]:
def detect_gustures(frame,landmarks_list,processed):
    if len(landmarks_list)>=21:

        index_finger_tip=find_finger_tip(processed)
        #print(index_finger_tip)
        thumb_index_dist= util.get_distance([landmarks_list[4],landmarks_list[5]])

        if thumb_index_dist is not None and thumb_index_dist < 100:
            angle=util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8])
            if angle>60:
                move_mouse(index_finger_tip)
            #move_mouse(index_finger_tip)
        #lift click
        elif is_left_click(landmarks_list,thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame, "Left Click",(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)


        #right click
        elif is_right_click(landmarks_list,thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame, "Right Click",(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)


        #double click
        elif is_double_click(landmarks_list,thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click",(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)


        #screenshot
        elif is_screenshot(landmarks_list,thumb_index_dist):
            im1=pyautogui.screenshot()
            label = random.randint(1,1000)
            im1.save(f'my_screenshot_{label}.png')
            cv2.putText(frame, "Screenshot Taken",(50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)


In [ ]:
def main():
    cap =cv2.VideoCapture(0)
    draw= mp.solutions.drawing_utils
    try:
        while cap.isOpened():
            ret,frame =cap.read()

            if not ret:
                break
            frame=cv2.flip(frame,1)
            frameRGB=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            processed=hands.process(frameRGB)

            landmarks_list=[]

            if processed.multi_hand_landmarks:
                hand_landmarks=processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame,hand_landmarks, mpHands.HAND_CONNECTIONS)
                
                for lm in hand_landmarks.landmark:
                    cx, cy = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                    cv2.circle(frame, (cx, cy), 5, (255, 0, 0), cv2.FILLED) 
                    landmarks_list.append((lm.x, lm.y))

                #print(landmarks_list)
            detect_gustures(frame,landmarks_list,processed)
            

            cv2.imshow('Frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()